In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow import keras
from keras.models import Sequential
from keras import layers, Model
from sklearn.model_selection import train_test_split

import numpy as np
from keras import Model
from keras.layers import Conv2D, PReLU,BatchNormalization, Flatten
from keras.layers import UpSampling2D, LeakyReLU, Dense, Input, add
from tqdm import tqdm

In [2]:
def res_block(input_dim):
    model = Conv2D(64, (3,3), padding = 'same' )(input_dim)
    model = BatchNormalization()(model)
    model = PReLU(shared_axes = [1,2])(model)
    model = Conv2D(64, (3,3), padding = 'same' )(model)
    model = BatchNormalization()(model)
    return add([input_dim, model])

In [3]:
def upscale_block(input_dim):
    model = Conv2D(256,(3,3), strides=1, padding = 'same')(input_dim)
    model = UpSampling2D(size = (2,2))(model)
    model = PReLU(shared_axes=[1, 2])(model)
    return model

In [4]:
def generator(input, res_range = 1,upscale_range=1):
    model = Conv2D(64,(9,9), strides=1, padding = 'same')(input)
    model = PReLU(shared_axes = [1,2])(model)
    model1 = model
    for i in range(res_range):
        model = res_block(model)
    model = Conv2D(64, (3,3), padding = 'same' )(model)
    model = BatchNormalization()(model)
    model = add([model,model1])
    for i in range(upscale_range):
        model  =upscale_block(model)
    output = Conv2D(3, (9,9),  padding='same')(model)
    return Model(input, output)

In [5]:
#code for building discriminator
def discrim_block(input_dim, fmaps = 64, strides = 1):
    model = Conv2D(fmaps, (3,3), padding = 'same', strides  = strides)(input_dim)
    model = BatchNormalization()(model)
    model = LeakyReLU()(model)
    return model

In [6]:
def discriminator(input):
    model = Conv2D(64,(3,3),padding='same')(input)
    model = LeakyReLU()(model)
    model = discrim_block(model, strides = 2)
    model = discrim_block(model, fmaps  = 128)
    model = discrim_block(model, fmaps = 128, strides = 2)
    model = discrim_block(model, fmaps=256)
    model = discrim_block(model, fmaps=256, strides=2)
    model = discrim_block(model, fmaps=512)
    model = discrim_block(model, fmaps=512, strides=2)
    model = Flatten()(model)
    model = Dense(1024)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    out = Dense(1, activation='sigmoid')(model)
    return Model(input, out)

In [7]:
from tensorflow.keras.applications.vgg19 import VGG19
def build_vgg(hr_shape):
    vgg = VGG19(weights="imagenet", include_top=False, input_shape=hr_shape)

    return Model(inputs=vgg.inputs, outputs=vgg.layers[10].output)

In [8]:
def create_comb(gen_model, disc_model, vgg, lr_ip, hr_ip):
    gen_img = gen_model(lr_ip)

    gen_features = vgg(gen_img)

    disc_model.trainable = False
    validity = disc_model(gen_img)

    return Model(inputs=[lr_ip, hr_ip], outputs=[validity, gen_features])

In [9]:
import os
from os import listdir
import gdown
from zipfile import ZipFile
import pickle
import cv2
from os import listdir

In [10]:


from tensorflow.keras.utils import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            img_lr = cv2.resize(image, (32,32))
            img_hr = cv2.resize(image, (128,128))
            return img_to_array(img_lr), img_to_array(img_hr)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [11]:
def getImgeList(directory_root):
    
    image_list_lr,image_list_hr, label_list = [], [], []
    try:
        print("[INFO] Loading images ...")
        root_dir = listdir(directory_root)
        #print(root_dir)
        for plant_folder in root_dir :
            # remove .DS_Store from list
            if plant_folder == ".DS_Store" :
                root_dir.remove(directory)

        for plant_folder in root_dir :
    #         print("on plant folder")
            plant_image_list = listdir(f"{directory_root}/{plant_folder}")
            for single_image in plant_image_list :

    #             print("in plant folder")
                if single_image == ".DS_Store" :
                    plant_image_list.remove(single_image)

            for image in plant_image_list[:6000]:
    #             print("getting img path")
                image_directory = f"{directory_root}/{plant_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
    #                 print("checking if img correct format")
                    lr_img, hr_img = convert_image_to_array(image_directory)
                    image_list_lr.append(lr_img)
                    image_list_hr.append(hr_img)
                    label_list.append(plant_folder)
        print("[INFO] Image loading completed")  
        return image_list_lr, image_list_hr, label_list
        
    except Exception as e:
        print(f"Error : {e}")

In [12]:
directory_root = 'DataB_strawberry'
image_list_lr,image_list_hr, label_list = getImgeList(directory_root)

[INFO] Loading images ...
[INFO] Image loading completed


In [13]:
from sklearn.preprocessing import LabelBinarizer
normalized_image_list_lr = np.array(image_list_lr, dtype = np.float16) / 127.5 - 1.
normalized_image_list_hr = np.array(image_list_hr, dtype = np.float16) / 127.5 - 1.
print(normalized_image_list_hr.shape)
# label_gen = LabelBinarizer()
# image_labels = label_gen.fit_transform(label_list)
# pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
# n_classes = len(label_gen.classes_)

(3327, 128, 128, 3)


In [14]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
hr_train, hr_test, lr_train, lr_test = train_test_split(normalized_image_list_hr, normalized_image_list_lr, test_size=0.1, random_state = 42) 
print(hr_train.shape)
print(lr_train.shape)
print(hr_test.shape)

making the training and testing split with 30% of the dataset as testing set
(2994, 128, 128, 3)
(2994, 32, 32, 3)
(333, 128, 128, 3)


In [15]:
hr_shape = (hr_train.shape[1], hr_train.shape[2], hr_train.shape[3])
lr_shape = (lr_train.shape[1], lr_train.shape[2], lr_train.shape[3])
print(hr_shape)
print(lr_shape)

lr_ip = Input(shape=lr_shape)
hr_ip = Input(shape=hr_shape)

(128, 128, 3)
(32, 32, 3)


In [16]:
generator = generator(lr_ip,res_range=16,upscale_range= 2)
generator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   15616       input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 32, 32, 64)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   36928       p_re_lu[0][0]                    
______________________________________________________________________________________________

In [17]:
discriminator = discriminator(hr_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_33 (Batc (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 64, 64, 128)       7385

In [18]:
vgg = build_vgg((128,128,3))
print(vgg.summary())
vgg.trainable = False

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [19]:
gan_model = create_comb(generator, discriminator, vgg, lr_ip, hr_ip)

In [20]:
# import tensorflow as tf

# def ssim_loss(y_true, y_pred):
#     return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)

# gan_model.compile(loss=[ssim_loss, "mse"], loss_weights=[1e-3, 1], optimizer="adam")

gan_model.compile(loss=["binary_crossentropy", "mse"], loss_weights=[1e-3, 1], optimizer="adam")
gan_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 128, 128, 3)  2044291     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 1)            38249281    model[0][0]                      
____________________________________________________________________________________________

In [21]:
batch_size = 1
train_lr_batches = []
train_hr_batches = []
for it in range(int(lr_train.shape[0] / batch_size)):
    start_idx = it * batch_size
    end_idx = start_idx + batch_size
    train_lr_batches.append(lr_train[start_idx:end_idx])
    train_hr_batches.append(hr_train[start_idx:end_idx])

In [22]:
#using VGG content loss
import keras.backend as K
from keras.losses import mean_squared_error
epochs = 3
#Enumerate training over epochs
for e in range(epochs):
    content_weight = 0.01
    fake_label = np.zeros((batch_size, 1)) # Assign a label of 0 to all fake (generated images)
    real_label = np.ones((batch_size,1)) # Assign a label of 1 to all real images.
    
    #Create empty lists to populate gen and disc losses. 
    g_losses = []
    d_losses = []
    
    #Enumerate training over batches. 
    for b in tqdm(range(len(train_hr_batches))):
        lr_imgs = train_lr_batches[b] #Fetch a batch of LR images for training
        hr_imgs = train_hr_batches[b] #Fetch a batch of HR images for training
        
        fake_imgs = generator.predict_on_batch(lr_imgs) #Fake images
        
        #First, train the discriminator on fake and real HR images. 
        discriminator.trainable = True
        d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
        d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        
        #Now, train the generator by fixing discriminator as non-trainable
        discriminator.trainable = False
        
        #Average the discriminator loss, just for reporting purposes. 
        d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
        #Extract VGG features, to be used towards calculating loss
        image_features = vgg.predict(hr_imgs)
     
        #Train the generator via GAN. 
        #Remember that we have 2 losses, adversarial loss and content (VGG) loss
        g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])
        
        # Calculate content loss using VGG features of generated images
        gen_features = vgg.predict(fake_imgs)
        content_loss = content_weight * mean_squared_error(image_features, gen_features)
        # Add content loss to generator loss
        g_loss += content_loss
        
        #Save losses to a list so we can average and report. 
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
    #Convert the list of losses to an array to make it easy to average    
    g_losses = np.array(g_losses)
    d_losses = np.array(d_losses)
    
    #Calculate the average losses for generator and discriminator
    g_loss = np.sum(g_losses, axis=0) / len(g_losses)
    d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
    #Report the progress during training. 
    print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

    if (e+1) % 10 == 0: #Change the frequency for model saving, if needed
        #Save the generator after every n epochs (Usually 10 epochs)
        generator.save("gen_e_"+ str(e+1) +".h5")


100%|██████████| 2994/2994 [14:07<00:00,  3.53it/s]


epoch: 1 g_loss: [[[65.168495 65.74567  66.21369  ... 66.68457  66.1483   65.38935 ]
  [65.51296  66.20262  66.68318  ... 67.44454  66.870964 65.9178  ]
  [65.785034 66.566826 67.09446  ... 68.28584  67.60481  66.41464 ]
  ...
  [66.733894 68.01024  68.962456 ... 75.21889  73.618965 70.51854 ]
  [66.08423  67.157646 68.06895  ... 73.81875  71.81543  69.04371 ]
  [65.42128  66.26919  67.02612  ... 71.4706   69.74731  67.40804 ]]] d_loss: [1.42228134 0.63360053]


100%|██████████| 2994/2994 [13:48<00:00,  3.61it/s]


epoch: 2 g_loss: [[[ 1981646.2   4530213.5   7692141.5  ...   595560.75   329886.
     147410.92]
  [ 3252556.5   6276489.5  10157165.   ...   827589.06   454401.97
     223904.64]
  [ 5044168.    9216059.   14044315.   ...  1280836.9    699293.5
     340894.47]
  ...
  [48469836.   63951372.   65506964.   ... 79323120.   54038496.
   31558440.  ]
  [40471492.   58973328.   69838024.   ... 90657960.   54707752.
   29059894.  ]
  [28046826.   47854388.   59997024.   ... 86762208.   52473016.
   24047502.  ]]] d_loss: [0.50589223 0.74549098]


100%|██████████| 2994/2994 [13:45<00:00,  3.63it/s]


epoch: 3 g_loss: [[[58.984726 59.059975 59.06111  ... 59.068157 59.067036 58.98436 ]
  [59.064907 59.16151  59.13734  ... 59.150635 59.17715  59.06848 ]
  [59.071217 59.156723 59.130783 ... 59.142864 59.17543  59.081444]
  ...
  [59.16209  59.309505 59.298676 ... 59.315136 59.331215 59.17762 ]
  [59.146496 59.3201   59.327263 ... 59.34501  59.348713 59.17447 ]
  [59.03275  59.16731  59.191273 ... 59.206802 59.195034 59.064926]]] d_loss: [0.11784588 0.95774883]


In [ ]:
# import keras.backend as K
# from keras.losses import MeanSquaredError
# from keras.applications.vgg19 import VGG19
# from keras.models import Model
# from keras_contrib.losses import DSSIMObjective

# # Set the weight for content loss
# content_weight = 0.01

# def srgan_loss(hr_image, sr_image, disc_generated_output):
#     # Define loss functions
#     mse_loss = MeanSquaredError()(hr_image, sr_image)
#     ssim_loss = DSSIMObjective()(hr_image, sr_image)

#     # Calculate adversarial loss
#     adversarial_loss = 1 - K.mean(disc_generated_output)

#     # Calculate content loss
#     vgg = VGG19(include_top=False, weights='imagenet')
#     vgg.trainable = False
#     content_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block5_conv4').output)
#     hr_features = content_model(hr_image)
#     sr_features = content_model(sr_image)
#     content_loss = content_weight * MeanSquaredError()(hr_features, sr_features)

#     # Combine the losses
#     total_loss = adversarial_loss + ssim_loss + content_loss

#     return total_loss


# epochs = 10
# #Enumerate training over epochs
# for e in range(epochs):
    
#     fake_label = np.zeros((batch_size, 1)) # Assign a label of 0 to all fake (generated images)
#     real_label = np.ones((batch_size,1)) # Assign a label of 1 to all real images.
    
#     #Create empty lists to populate gen and disc losses. 
#     g_losses = []
#     d_losses = []
    
#     #Enumerate training over batches. 
#     for b in tqdm(range(len(train_hr_batches))):
#         lr_imgs = train_lr_batches[b] #Fetch a batch of LR images for training
#         hr_imgs = train_hr_batches[b] #Fetch a batch of HR images for training
        
#         fake_imgs = generator.predict_on_batch(lr_imgs) #Fake images
        
#         #First, train the discriminator on fake and real HR images. 
#         discriminator.trainable = True
#         d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
#         d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        
#         #Now, train the generator by fixing discriminator as non-trainable
#         discriminator.trainable = False
        
#         #Average the discriminator loss, just for reporting purposes. 
#         d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
#         #Train the generator via GAN. 
#         #Remember that we have 3 losses, adversarial loss, content (VGG) loss and SSIM loss
#         g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, hr_imgs, hr_imgs])
        
#         #Save losses to a list so we can average and report. 
#         d_losses.append(d_loss)
#         g_losses.append(g_loss)
        
#     #Convert the list of losses to an array to make it easy to average    
#     g_losses = np.array(g_losses)
#     d_losses = np.array(d_losses)
    
#     #Calculate the average losses for generator and discriminator
#     g_loss = np.sum(g_losses, axis=0) / len(g_losses)
#     d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
#     #Report the progress during training. 
#     print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

#     if (e+


In [65]:
# usign ssim loss
# from skimage import structural_similarity as ssim

# epochs = 4

# #Enumerate training over epochs
# for e in range(epochs):
    
#     fake_label = np.zeros((batch_size, 1)) # Assign a label of 0 to all fake (generated images)
#     real_label = np.ones((batch_size,1)) # Assign a label of 1 to all real images.
    
#     #Create empty lists to populate gen and disc losses. 
#     g_losses = []
#     d_losses = []
    
#     #Enumerate training over batches. 
#     for b in tqdm(range(len(train_hr_batches))):
#         lr_imgs = train_lr_batches[b] #Fetch a batch of LR images for training
#         hr_imgs = train_hr_batches[b] #Fetch a batch of HR images for training
        
#         fake_imgs = generator.predict_on_batch(lr_imgs) #Fake images
        
#         #First, train the discriminator on fake and real HR images. 
#         discriminator.trainable = True
#         d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
#         d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        
#         #Now, train the generator by fixing discriminator as non-trainable
#         discriminator.trainable = False
        
#         #Average the discriminator loss, just for reporting purposes. 
#         d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
#         #Calculate SSIM loss between HR images and generated images
#         ssim_loss = np.mean([ssim(hr_imgs[i], fake_imgs[i], multichannel=True) for i in range(batch_size)])
     
#         #Train the generator via GAN. 
#         #Remember that we have 2 losses, adversarial loss and SSIM loss
#         g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, ssim_loss])
        
#         #Save losses to a list so we can average and report. 
#         d_losses.append(d_loss)
#         g_losses.append(g_loss)
        
#     #Convert the list of losses to an array to make it easy to average    
#     g_losses = np.array(g_losses)
#     d_losses = np.array(d_losses)
    
#     #Calculate the average losses for generator and discriminator
#     g_loss = np.sum(g_losses, axis=0) / len(g_losses)
#     d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
#     #Report the progress during training. 
#     print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

#     if (e+1) % 10 == 0: #Change the frequency for model saving, if needed
#         #Save the generator after every n epochs (Usually 10 epochs)
#         generator.save("gen_e_"+ str(e+1) +".h5")


In [82]:
# orignal
epochs = 1
#Enumerate training over epochs
for e in range(epochs):
    
    fake_label = np.zeros((batch_size, 1)) # Assign a label of 0 to all fake (generated images)
    real_label = np.ones((batch_size,1)) # Assign a label of 1 to all real images.
    
    #Create empty lists to populate gen and disc losses. 
    g_losses = []
    d_losses = []
    
    #Enumerate training over batches. 
    for b in tqdm(range(len(train_hr_batches))):
        lr_imgs = train_lr_batches[b] #Fetch a batch of LR images for training
        hr_imgs = train_hr_batches[b] #Fetch a batch of HR images for training
        
        fake_imgs = generator.predict_on_batch(lr_imgs) #Fake images
        
        #First, train the discriminator on fake and real HR images. 
        discriminator.trainable = True
        d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
        d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        
        #Now, train the generator by fixing discriminator as non-trainable
        discriminator.trainable = False
        
        #Average the discriminator loss, just for reporting purposes. 
        d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
        #Extract VGG features, to be used towards calculating loss
        image_features = vgg.predict(hr_imgs)
     
        #Train the generator via GAN. 
        #Remember that we have 2 losses, adversarial loss and content (VGG) loss
        g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])
        
        #Save losses to a list so we can average and report. 
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
    #Convert the list of losses to an array to make it easy to average    
    g_losses = np.array(g_losses)
    d_losses = np.array(d_losses)
    
    #Calculate the average losses for generator and discriminator
    g_loss = np.sum(g_losses, axis=0) / len(g_losses)
    d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
    #Report the progress during training. 
    print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

    if (e+1) % 10 == 0: #Change the frequency for model saving, if needed
        #Save the generator after every n epochs (Usually 10 epochs)
        generator.save("gen_e_"+ str(e+1) +".h5")

  0%|          | 0/2994 [00:00<?, ?it/s]


ValueError: in user code:

    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\keras\engine\traini

In [ ]:
from tensorflow.keras.utils import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            img_gan = cv2.resize(image, (32,32))
            return img_to_array(img_gan)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
directory_root = 'DCGAN_tomato_bacterial_final'
image_list_GAN, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    #print(root_dir)
    for plant_folder in root_dir :
        # remove .DS_Store from list
        if plant_folder == ".DS_Store" :
            root_dir.remove(directory)
            

    for plant_folder in root_dir :
#         print("on plant folder")
        plant_image_list = listdir(f"{directory_root}/{plant_folder}")
        
     
        
        for single_image in plant_image_list :
            
#             print("in plant folder")
            if single_image == ".DS_Store" :
                plant_image_list.remove(single_image)

        for image in plant_image_list[:2000]:
#             print("getting img path")
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".png") == True or image_directory.endswith(".PNG") == True:
#                 print("checking if img correct format")
                gan_img = convert_image_to_array(image_directory)
                image_list_GAN.append(gan_img)
                label_list.append(plant_folder)
                    
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Image loading completed


In [5]:
normalized_image_list_GAN = np.array(image_list_GAN, dtype = np.float16) / 255.0

In [6]:
print(normalized_image_list_GAN.shape)

(2000, 32, 32, 3)


In [23]:
generator.compiled_metrics == None

True

In [31]:
generator.compile()
generator.save('generator_leafScorch_strawberry_SRGAN.h5')

In [38]:
loaded_gen = keras.models.load_model('generator_leafScorch_strawberry_SRGAN6.h5')

In [39]:
test_results = loaded_gen.predict(lr_test)

In [40]:
from keras.preprocessing.image import array_to_img
# test_results = test_results * 255.0
test_results = 0.5 * test_results + 0.5
for i in range(len(test_results)):
    img = array_to_img((test_results[i, :,:,:]))
    img.save(os.path.join('SRGAN_gen',f'generated_img_epoch_{i}.png')) 

In [9]:
def split_list(lst,parts):
    n = len(lst)
    n_thirds = n // parts
    remainder = n % parts
    
    parts_list = []
    start = 0
    
    for i in range(3):
        end = start + n_thirds
        
        if i < remainder:
            end += 1
        
        parts_list.append(lst[start:end])
        start = end
    
    return parts_list

In [10]:
partitions = split_list(normalized_image_list_GAN, 3)
part1 = partitions[0]
part2 = partitions[1]
part3 = partitions[2]
print(normalized_image_list_GAN.shape)

(2000, 32, 32, 3)


In [11]:
from random import randint
from keras.preprocessing.image import array_to_img


def predict_parts(gen_model, part_list):
    generated_imgs = loaded_gen.predict(part_list)
    generated_imgs = generated_imgs * 255.0
    return generated_imgs

def save_to_folder(l1,l2,l3):
    check_point = 0
    for i in range(len(l1)):
        img = array_to_img((l1[i, :,:,:]))
        img.save(os.path.join('SRGAN_X_DCGAN_tomato_bacterial',f'generated_img_epoch_{i}.png')) 
        check_point = i
    for i in range(len(l2)):
        img = array_to_img((l2[i, :,:,:]))
        img.save(os.path.join('SRGAN_X_DCGAN_tomato_bacterial',f'generated_img_epoch_{check_point + i}.png')) 
        check_point = i
    for i in range(len(l3)):
        img = array_to_img((l3[i, :,:,:]))
        img.save(os.path.join('SRGAN_X_DCGAN_tomato_bacterial',f'generated_img_epoch_{check_point + i}.png')) 
        check_point = i

In [12]:
gen_imgs1 = predict_parts(loaded_gen, part1)
gen_imgs2 = predict_parts(loaded_gen, part2)
gen_imgs3 = predict_parts(loaded_gen, part3)


In [13]:
save_to_folder(gen_imgs1, gen_imgs2, gen_imgs3)